# Differential Power Analysis

This notebook demonstrates the functionality of the `DPA` class, which implements Differential Power Analysis side-channel attack. The attack is first performed and evaluated on leakage traces simulated by the `LeakageTarget` class and subsequently on leakage traces simulated by the `EmulatorTarget` class.

## Initialisation

In [ ]:
from pyecsca.ec.mult import LTRMultiplier
from pyecsca.ec.mod import Mod
from pyecsca.ec.point import Point, InfinityPoint
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.curve import EllipticCurve
from pyecsca.ec.params import DomainParameters
from pyecsca.sca.attack.DPA import DPA
from pyecsca.sca.attack.CPA import CPA
from pyecsca.sca.attack.leakage_model import HammingWeight
from pyecsca.sca.target.leakage import LeakageTarget
from pyecsca.sca.trace import Trace
from pyecsca.sca.trace.edit import stretch
from pyecsca.sca.trace.process import rolling_mean
from pyecsca.codegen.client import EmulatorTarget
from rainbow import TraceConfig
from rainbow.leakage_models import HammingWeight
from random import randint
import holoviews as hv
import multiprocessing as mp
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
hv.extension("bokeh")


We first define the elliptic curve parameters we are going to be using for the demonstration.

In [ ]:
model = ShortWeierstrassModel()
coords = model.coordinates["projective"]
p = 0xd7d1247f
a = Mod(0xa4a44016, p)
b = Mod(0x73f76716, p)
n = 0xd7d2a475
h = 1
gx, gy, gz = Mod(0x54eed6d7, p), Mod(0x6f1e55ac, p), Mod(1, p)
generator = Point(coords, X=gx, Y=gy, Z=gz)
neutral = InfinityPoint(coords)

curve = EllipticCurve(model, coords, p, neutral, {"a": a, "b": b})
params = DomainParameters(curve, generator, n, h)

add = coords.formulas["add-1998-cmo"]
dbl = coords.formulas["dbl-1998-cmo"]
scl = coords.formulas["z"]

mult = LTRMultiplier(add, dbl, None)

We create and initialize an instance of the `LeakageTarget` class with the above elliptic curve parameters.

In [ ]:
target = LeakageTarget(model, coords, mult, HammingWeight())
target.set_params(params)

Using the attack, we will try to recover an 8-bit scalar used in the scalar multiplication operation.

In [ ]:
scalar_bit_length = 8
secret_scalar = 229

We will perform scalar multiplication on 10 000 random points on the curve and simulate 10 000 corresponding leakage traces, which we will then use to evaluate the attack. 

In [ ]:
num_of_traces = 2000

In [ ]:
generated_points, generated_traces = target.simulate_scalar_mult_traces(num_of_traces, secret_scalar)

## Performing the attack

We create and initialize an instance of the `DPA` class containing the attack's implementation.

In [ ]:
mult.init(params, params.generator)
real_pub_key = mult.multiply(secret_scalar)

In [ ]:
dpa = DPA(generated_points, generated_traces, mult, params)

We perform the attack and test that the scalar we recovered is correct.

In [ ]:
recovered_scalar = dpa.perform(scalar_bit_length, real_pub_key)
print(recovered_scalar)
print(recovered_scalar == secret_scalar)

We can visualize the differences of means calculated during the attack. Below are shown DOMs for correct and incorrect guess of the second bit of the scalar, followed by DOMs of correct guesses for fourth and sixth bit of the scalar to show the location of peeks.

In [ ]:
# Correct difference of means when guessing 2. bit (guessed scalar 192 = 1100 0000)
dpa.plot_difference_of_means(dpa.doms['guess_one'][0])

In [ ]:
# Incorrect difference of means when guessing 2. bit (guessed scalar 128 = 1000 0000)
dpa.plot_difference_of_means(dpa.doms['guess_zero'][0])

In [ ]:
# Correct difference of means when guessing 4. bit (guessed scalar 224 = 1110 0000)
dpa.plot_difference_of_means(dpa.doms['guess_zero'][2])

In [ ]:
# Correct difference of means when guessing 6. bit (guessed scalar 229 = 1110 0100)
dpa.plot_difference_of_means(dpa.doms['guess_one'][-2])

## Evaluating the effectiveness of the attack

We will evaluate how the success rate of the attack increases with an increasing number of traces and subsequently evaluate how the success rate for a fixed number of traces changes with increasing noise. 

To have a convenient way of generating random subsets, we define the following function:

In [ ]:
def select_random_sample(size, points, traces):
    sample_points = []
    sample_traces = []
    selected_numbers = []
    for _ in range(size):
        num = randint(0, num_of_traces - 1)
        if num not in selected_numbers:
            selected_numbers.append(num)
        sample_points.append(points[num])
        sample_traces.append(traces[num])
    return sample_points, sample_traces

### Success rate with changing number of traces

We take subsets of different sizes and plot the success rate of the DPA corresponding to them. For each size, we perform the DPA 100 times on 100 generated subsets.

In [ ]:
def DPA_traces(points, traces, sample_size, scalar_length, pub_key, multiplier, domain_params):
    sample_points, sample_traces = select_random_sample(sample_size, points, traces)
    sample_dpa = DPA(sample_points, sample_traces, multiplier, domain_params)
    recovered = sample_dpa.perform(scalar_length, pub_key)
    return recovered == secret_scalar


In [ ]:
number_of_subsets = 17
successes_rates_traces = []
number_of_samples = [x * 100 for x in range(1, number_of_subsets)]
for num in number_of_samples:
    with mp.Pool() as pool:
        result_objects = [pool.apply_async(DPA_traces, args=(generated_points, generated_traces, num, scalar_bit_length, 
                                                             real_pub_key, mult, params)) for _ in range(100)]
        results = [result.get() for result in result_objects]
    successes_rates_traces.append(sum(results) / 100)
    print(f"done for {num}")

We plot the results.

In [ ]:
plt.plot(number_of_samples, successes_rates_traces, color='black')
plt.ylim([0, 1])
plt.xlabel('number of traces')
plt.ylabel('DPA success rate')

### Success rate with fixed number of traces and changing noise

As we now know the the minimum amount of traces needed for 100% success rate, we generate 100 subsets with that amount of traces and perform DPA for each subset. We do this multiple times with different amount of noise.

In [ ]:
def normalize_trace(trace: Trace):
    min_val = np.min(trace.samples)
    max_val = np.max(trace.samples)
    scaled_samples = (trace.samples - min_val) / (max_val - min_val) 
    trace.samples = scaled_samples   

def DPA_noise(points, traces, sample_size, standard_deviation, scalar_length, pub_key, multiplier, domain_params):
    sample_points, sample_traces = select_random_sample(sample_size, points, traces)
    for trace in sample_traces:
        normalize_trace(trace)
    noise = np.random.normal(0, standard_deviation, sample_traces[0].samples.shape)
    for trace in sample_traces:
        trace.samples = trace.samples + noise
    sample_dpa = DPA(sample_points, sample_traces, multiplier, domain_params)
    recovered = sample_dpa.perform(scalar_length, pub_key)
    return recovered == secret_scalar


In [ ]:
number_of_subsets = 17
successes_rates_noise = []
sample_size = 1600
noise_sds = [0.05 * x for x in range(number_of_subsets)] 
for sd in noise_sds:
    with mp.Pool() as pool:
        result_objects = [pool.apply_async(DPA_noise, args=(generated_points, generated_traces, sample_size, sd, scalar_bit_length, 
                                                             real_pub_key, mult, params)) for _ in range(100)]
        results = [result.get() for result in result_objects]
    successes_rates_noise.append(sum(results) / 100)
    print(f"done for {sd}")

We plot the results.

In [ ]:
plt.plot(noise_sds, successes_rates_noise, color='black')
plt.ylim([0, 1])
plt.xlabel('Gaussian noise (standard deviation)')
plt.ylabel('DPA success rate')

## DPA on leakage traces simulated by `EmulatorTarget`.

We gather leakage traces simulated by the `EmulatorTarget`.

In [ ]:
with open('/data/xbatora/traces_8bit.pickle', 'rb') as f:
    traces_emulator = pickle.load(f)

In [ ]:
points_emulator = []
max_length = 0
for trace in traces_emulator:
    if(len(trace) > max_length):
        max_length = len(trace)
    points_emulator.append(Point(coords, **trace.meta['coords']))

The traces are not equally long. We `stretch` them to the same length. This should also reduce misalignment.

In [ ]:
stretched_traces = []
for trace in traces_emulator:
    stretched_traces.append(stretch(trace, max_length))

We recover the second bit of the scalar. We know it should equal one, so we plot its corresponding DOM that should show a peak.

In [ ]:
recovered_scalar = 128 #1000 0000

In [ ]:
dpa_emulator = DPA(points_emulator, stretched_traces, mult, params)

In [ ]:
recovered_scalar = 128 #1000 0000
res = dpa_emulator.recover_bit(recovered_scalar, 1, scalar_bit_length, real_pub_key)

The peak is not visible

In [ ]:
dpa_emulator.plot_difference_of_means(dpa_emulator.doms['guess_one'][0])

The peak is not visible due to a significant misalignment. We apply rolling mean to the traces, which should reduce the misalignment. We perform the DPA again.

In [ ]:
rolling_mean_traces = []
for trace in stretched_traces:
    rolling_mean_traces.append(rolling_mean(trace, 3000))

In [ ]:
dpa_emulator = DPA(points_emulator, rolling_mean_traces, mult, params)

In [ ]:
recovered_scalar = 128 #1000 0000
res = dpa_emulator.recover_bit(recovered_scalar, 1, scalar_bit_length, real_pub_key)
print(res)

In [ ]:
dpa_emulator.plot_difference_of_means(dpa_emulator.doms['guess_one'][0])